In [1]:
# custom_objects downloading
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
import numpy as np

@tf.keras.utils.register_keras_serializable()
class RepeatChannels(Layer):
    def __init__(self, **kwargs):
        super(RepeatChannels, self).__init__(**kwargs)

    def call(self, inputs):
        # Assuming the repeat operation repeats across channels
        return K.repeat_elements(inputs, rep=3, axis=-1)

    def get_config(self):
        config = super(RepeatChannels, self).get_config()
        return config

@tf.keras.utils.register_keras_serializable()
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = K.sum(y_true * y_pred)
        union = K.sum(y_true) + K.sum(y_pred) - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

@tf.keras.utils.register_keras_serializable()
def dice_coef(y_true, y_pred, smooth=1e-6):
    """
    Compute the Dice coefficient, a measure of overlap between two samples.
    """
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)

    intersection = tf.reduce_sum(y_true * y_pred)
    dice = (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
    return dice

@tf.keras.utils.register_keras_serializable()
def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

@tf.keras.utils.register_keras_serializable()
def bce_dice_loss(y_true, y_pred):
    # Binary Cross-Entropy
    bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    # Dice Loss
    intersection = tf.reduce_sum(y_true * y_pred)
    dice_loss = 1 - (2. * intersection + 1) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + 1)
    # Combine both losses
    return bce_loss + dice_loss

# Load models with custom objects
from tensorflow.keras.models import load_model

custom_objects = {
    'RepeatChannels': RepeatChannels,
    'iou': iou,
    'dice_coef': dice_coef,
    'dice_loss': dice_loss,
    'bce_dice_loss': bce_dice_loss
}

/Users/ammaster10/Documents/Github/Year4/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
import os
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from pycocotools.coco import COCO

# This is your existing noise function
def preprocess_noise(image, method='median', kernel_size=3):
    if kernel_size % 2 == 0 or kernel_size < 1:
        raise ValueError("Kernel size must be an odd integer greater than 1.")
    if method == 'median':
        return cv2.medianBlur(image, kernel_size)
    raise ValueError("Unsupported denoising method.")

# --- THIS IS THE MODIFIED FUNCTION ---
def run_inference_and_save_outputs(model, image_dir, coco_json_path, binary_mask_dir, prob_map_dir, target_size=(512, 512)):
    """
    Runs model prediction and saves BOTH the binary mask and the probability map.

    Args:
        model: Trained Keras model.
        image_dir (str): Base directory where image subfolders are located.
        coco_json_path (str): Path to the COCO JSON file.
        binary_mask_dir (str): Directory to save the final .png binary masks.
        prob_map_dir (str): Directory to save the raw .npy probability maps.
        target_size (tuple): The (Height, Width) for the model.
    """
    # --- MODIFIED: Create both output directories ---
    os.makedirs(binary_mask_dir, exist_ok=True)
    os.makedirs(prob_map_dir, exist_ok=True)

    coco = COCO(coco_json_path)
    img_ids = coco.getImgIds()
    print(f"Found {len(img_ids)} images. Saving PNGs to '{binary_mask_dir}' and NPYs to '{prob_map_dir}'...")

    for img_id in tqdm(img_ids, desc="Generating predictions"):
        img_info = coco.loadImgs(img_id)[0]
        filename_from_json = img_info['file_name']
        image_path = os.path.join(image_dir, filename_from_json)
        
        try:
            if not os.path.exists(image_path):
                print(f"\nWarning: File not found, skipping -> {image_path}")
                continue
            
            # --- Preprocessing (unchanged) ---
            img = load_img(image_path, color_mode='grayscale', target_size=target_size)
            img_array = img_to_array(img)
            img_uint8 = img_array.astype(np.uint8)
            img_denoised = preprocess_noise(img_uint8, method='median', kernel_size=3)
            img_normalized = img_denoised.astype(np.float32) / 255.0
            model_input = np.expand_dims(img_normalized, axis=0)

            # --- PREDICTION AND SAVING ---
            
            # 1. Get the raw prediction. This is your probability map!
            # Shape is (1, 512, 512, 1). Values are floats from 0.0 to 1.0.
            raw_prediction = model.predict(model_input, verbose=0)
            
            # 2. Squeeze the extra dimensions to get a clean (512, 512) array.
            # Safely squeeze from (1, H, W, 1) → (H, W)
            if raw_prediction.shape == (1, 512, 512, 1):
                probability_map = raw_prediction[0, :, :, 0]
            else:
                raise ValueError(f"Unexpected prediction shape: {raw_prediction.shape}")
            

            # 3. --- NEW: Save the probability map as a .npy file ---
            base_name = os.path.splitext(os.path.basename(filename_from_json))[0]
            prob_map_output_path = os.path.join(prob_map_dir, f"{base_name}_prob_map.npy")
            np.save(prob_map_output_path, probability_map)

            # 4. --- NOW create and save the binary mask like before ---
            binary_mask = (probability_map > 0.5).astype(np.uint8) * 255
            output_image = Image.fromarray(binary_mask)
            
            binary_mask_output_path = os.path.join(binary_mask_dir, f"{base_name}_predicted_mask.png")
            output_image.save(binary_mask_output_path)

        except Exception as e:
            print(f"\nError processing {filename_from_json}: {e}")

    print(f"\n✅ Done! All predictions and probability maps saved.")

# --- HOW TO RUN THE UPDATED SCRIPT ---
if __name__ == '__main__':
    base_dir = "/Users/ammaster10/Documents/Github/Year4/CNVresearch/Validation/340_Folder"
    
    # Define all your paths
    image_dir = os.path.join(base_dir, "340_Test")
    coco_json_path = os.path.join(base_dir, "subset_340.json")
    
    # --- MODIFIED: Define separate output folders ---
    binary_mask_output_dir = os.path.join(base_dir, "predicted_masks_png")
    prob_map_output_dir = os.path.join(base_dir, "predicted_prob_maps_npy")
    
    MODEL_PATH = "/Users/ammaster10/Documents/Github/Year4/CNVresearch/Models/deeplabv3_plus_ultra_model_2025_02_10_01_37_48.keras" # !!! IMPORTANT !!! - Update this path

    try:
        model = load_model(MODEL_PATH, compile=False, custom_objects=custom_objects)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Error loading model: {e}")
        # Dummy model for testing
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import Conv2D
        model = Sequential([Conv2D(1, (1,1), activation='sigmoid', input_shape=(512,512,1))])
        print("Continuing with a dummy model.")

    run_inference_and_save_outputs(
        model=model,
        image_dir=image_dir,
        coco_json_path=coco_json_path,
        binary_mask_dir=binary_mask_output_dir, # Pass the new PNG dir
        prob_map_dir=prob_map_output_dir,       # Pass the new NPY dir
        target_size=(512, 512)
    )

Model loaded successfully.
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Found 340 images. Saving PNGs to '/Users/ammaster10/Documents/Github/Year4/CNVresearch/Validation/340_Folder/predicted_masks_png' and NPYs to '/Users/ammaster10/Documents/Github/Year4/CNVresearch/Validation/340_Folder/predicted_prob_maps_npy'...


Generating predictions: 100%|██████████| 340/340 [01:53<00:00,  3.01it/s]


✅ Done! All predictions and probability maps saved.


In [6]:
arr = np.load("/Users/ammaster10/Documents/Github/Year4/CNVresearch/Validation/340_Folder/predicted_prob_maps_npy/CNV-53264-13_prob_map.npy")
print(arr.shape, arr.min(), arr.max())

(512, 512) 1.4298232e-06 0.9999998
